## Download

In [1]:
### !pip install theohe-epias
### link: https://pypi.org/project/theohe-epias/

## Import and load Transparency WebService (WS)

In [2]:
## !pip install dataframe-image

In [3]:
from theohe_epias.transparency.service import WebServiceTransparency
import warnings
import pandas as pd
import dataframe_image as dfi
pd.set_option('display.max_columns', 24)
pd.set_option('display.max_rows', 24)


warnings.simplefilter("ignore")

ws = WebServiceTransparency()


## Download MCP | SMP | Direction

In [4]:
dataframe = ws.reports.mcp_smp_imbalance(
    startDate = (2021,1,1),
    endDate = (2023,12,31),function = "export")

In [5]:
dataframe

,Tarih,Saat,PTF (TL/MWh),SMF (TL/MWh),Pozitif Dengesizlik Fiyatı (TL/MWh),Negatif Dengesizlik Fiyatı (TL/MWh),SMF Yön
0,2021-01-01 00:00:00,2021-01-01 00:00:00,263.99,269.34,256.0703,277.4202,Enerji Açığı
1,2021-01-01 01:00:00,2021-01-01 01:00:00,236.99,258.34,229.8803,266.0902,Enerji Açığı
2,2021-01-01 02:00:00,2021-01-01 02:00:00,216.70,248.34,210.1990,255.7902,Enerji Açığı
3,2021-01-01 03:00:00,2021-01-01 03:00:00,218.07,248.34,211.5279,255.7902,Enerji Açığı
4,2021-01-01 04:00:00,2021-01-01 04:00:00,208.99,247.99,202.7203,255.4297,Enerji Açığı
...,...,...,...,...,...,...,...
26275,2023-12-31 19:00:00,2023-12-31 19:00:00,2499.67,2690.00,2424.6799,2770.7000,Enerji Açığı
26276,2023-12-31 20:00:00,2023-12-31 20:00:00,2472.34,1852.00,1796.4400,2546.5102,Enerji Fazlası
26277,2023-12-31 21:00:00,2023-12-31 21:00:00,2472.33,1200.00,1164.0000,2546.4999,Enerji Fazlası
26278,2023-12-31 22:00:00,2023-12-31 22:00:00,1800.00,630.00,611.1000,1854.0000,Enerji Fazlası


## Format Dataframe

In [6]:
def add_date_info(df):
    df["year"] = df["Tarih"].dt.year
    df["weekday"] = df["Tarih"].dt.weekday
    df["month"] = df["Tarih"].dt.month
    df["hour"] = df["Tarih"].dt.hour
    df["day_of_year"] = df["Tarih"].dt.day_of_year
    
    df["weekend"] = df["weekday"].apply(lambda x: 1 if x in [5,6] else 0)




add_date_info(dataframe)

df = dataframe[['year', 'month','hour','PTF (TL/MWh)', 'SMF (TL/MWh)', 'SMF Yön', ]]
df["SMF Yön"] = df["SMF Yön"].replace({'Enerji Fazlası':1, 'Enerji Açığı':-1, 'Dengede':0})

df = df.rename(columns = {
    "year":"Yıl",
    "month":"Ay",
    "hour":"Saat",
})



## Market Clearing Price

In [7]:
res_mcp = df.groupby(['Yıl', 'Ay','Saat',])['PTF (TL/MWh)'].mean().unstack("Saat")


### 2021

In [8]:
mcp2021 = (
    res_mcp.loc[2021].reset_index().set_index("Ay").T
).style.background_gradient(
    axis=0
).format("{:,.1f}").set_caption('<br>2021 yılı Piyasa Takas Fiyatı (PTF)<br>Ay Bazında Saatlik Ortalamalar')

mcp2021


Ay,1,2,3,4,5,6,7,8,9,10,11,12
Saat,,,,,,,,,,,,
0,307.4,305.8,307.3,306.5,363.8,380.5,586.5,584.8,500.8,580.7,669.3,987.3
1,289.0,277.1,303.5,321.4,362.7,362.9,531.0,574.9,490.1,700.3,864.6,898.7
2,278.0,258.3,289.7,313.6,358.0,354.0,460.2,515.5,452.4,576.1,689.6,807.8
3,262.9,249.0,275.2,305.3,353.2,344.8,425.0,490.3,460.2,526.7,652.8,762.0
4,258.9,246.0,274.7,304.4,349.9,348.3,418.6,478.9,438.5,500.4,640.9,758.9
5,261.1,256.5,284.4,307.3,341.2,333.5,374.2,449.0,437.6,542.5,675.4,782.8
6,269.3,271.3,300.3,273.5,303.9,321.8,364.1,398.5,431.4,607.9,723.5,844.7
7,278.5,271.9,294.8,284.0,311.3,337.8,373.2,433.1,432.8,613.3,747.2,895.8
8,291.8,299.0,318.0,304.9,330.2,376.4,483.5,517.1,489.2,730.0,"1,013.3","1,054.3"


### 2022

In [9]:
mcp2022 = (
    res_mcp.loc[2022].reset_index().set_index("Ay").T
).style.background_gradient(
    axis=0
).format("{:,.1f}").set_caption('<br>2022 yılı Piyasa Takas Fiyatı (PTF)<br>Ay Bazında Saatlik Ortalamalar')

mcp2022

Ay,1,2,3,4,5,6,7,8,9,10,11,12
Saat,,,,,,,,,,,,
0,"1,158.4","1,399.4","1,657.8","2,126.4","2,165.4","2,522.0","2,601.1","3,324.8","4,346.9","3,674.1","3,396.2","3,313.0"
1,"1,088.2","1,356.8","1,620.7","1,878.8","1,743.3","2,345.5","2,310.3","2,942.4","3,814.3","3,134.1","2,795.6","2,970.2"
2,"1,041.4","1,321.1","1,575.7","1,611.1","1,411.6","2,248.9","2,177.6","2,734.2","3,480.7","2,687.8","2,318.0","2,672.7"
3,"1,020.1","1,290.6","1,557.0","1,774.8","1,399.8","2,183.1","2,179.7","2,660.3","3,517.1","2,619.3","2,159.7","2,462.5"
4,950.6,"1,277.0","1,561.4","1,864.0","1,570.4","2,246.8","2,126.0","2,661.8","3,387.9","2,214.1","1,951.3","2,623.1"
5,973.3,"1,316.0","1,606.9","1,753.3","1,579.0","2,125.9","2,016.3","2,517.3","3,224.0","2,617.4","2,282.6","2,937.0"
6,"1,064.9","1,344.6","1,662.9","1,363.6","1,144.5","1,847.6","1,799.4","2,437.2","3,356.9","3,065.7","2,697.0","3,219.4"
7,"1,114.8","1,382.7","1,691.7","1,214.8","1,207.4","1,962.8","1,713.8","2,443.3","3,330.9","2,977.5","3,095.5","3,592.2"
8,"1,254.5","1,476.0","1,688.8","1,857.6","1,751.7","2,241.1","2,253.8","2,816.7","3,870.2","3,997.8","4,149.6","4,429.3"


### 2023

In [10]:
mcp2023 = (
    res_mcp.loc[2023].reset_index().set_index("Ay").T
).style.background_gradient(
    axis=0
).format("{:,.1f}").set_caption('<br>2023 yılı Piyasa Takas Fiyatı (PTF)<br>Ay Bazında Saatlik Ortalamalar')

mcp2023

Ay,1,2,3,4,5,6,7,8,9,10,11,12
Saat,,,,,,,,,,,,
0,"3,103.7","2,699.4","1,849.0","2,071.1","2,215.4","1,885.0","2,221.3","2,436.8","2,202.4","2,344.2","1,945.8","2,003.5"
1,"3,023.7","2,615.1","1,883.4","1,805.2","2,008.8","1,757.5","2,064.0","2,249.4","1,998.0","2,253.5","1,793.4","1,885.9"
2,"2,872.1","2,365.2","1,847.2","1,528.9","1,791.9","1,572.3","1,940.7","2,063.7","1,830.6","2,076.2","1,691.4","1,659.5"
3,"2,770.1","2,287.7","1,821.5","1,463.6","1,687.5","1,547.6","1,882.3","1,902.0","1,746.2","1,937.4","1,585.2","1,530.8"
4,"2,791.0","2,248.1","1,928.5","1,632.1","1,716.7","1,564.0","1,851.9","1,898.5","1,678.9","1,883.5","1,559.5","1,576.3"
5,"2,942.3","2,236.1","2,012.4","1,692.0","1,722.3","1,467.7","1,723.1","1,898.4","1,808.0","2,051.9","1,662.6","1,787.4"
6,"3,112.4","2,494.3","2,009.9","1,612.6","1,465.3","1,302.0","1,699.0","1,859.3","1,795.0","2,285.0","1,964.9","2,082.7"
7,"3,309.8","2,584.1","1,859.6","1,271.7","1,361.6","1,193.9","1,518.8","1,687.8","1,779.8","2,275.3","2,035.8","2,160.8"
8,"3,984.3","3,143.4","2,592.6","1,796.8","1,979.6","1,563.5","1,962.6","2,285.4","2,038.0","2,513.6","2,401.0","2,452.8"


## System Direction

In [11]:
res_smp_direction = df.groupby(['Yıl', 'Ay','Saat'])['SMF Yön'].mean().unstack("Saat")


### 2021

In [12]:
direction2021 = (
    100*res_smp_direction.loc[2021].reset_index().set_index("Ay").T
).style.background_gradient(
    cmap="RdYlGn",axis=None
).format("{:,.1f}").set_caption('<br>2021 yılı Sistem Yönü Ay Bazında Saatlik Ortalamalar<br>Enerji Fazlası: 100,<br> Dengede: 0<br> Enerji Açığı: -100')

direction2021

Ay,1,2,3,4,5,6,7,8,9,10,11,12
Saat,,,,,,,,,,,,
0,-41.9,-14.3,-41.9,6.7,-48.4,-66.7,-74.2,-22.6,73.3,-9.7,-40.0,16.1
1,-48.4,-21.4,-45.2,0.0,-45.2,-60.0,-54.8,-51.6,43.3,-51.6,-60.0,-22.6
2,-41.9,-21.4,-41.9,-16.7,-54.8,-53.3,-54.8,-19.4,46.7,0.0,-26.7,-29.0
3,-48.4,-17.9,-38.7,-6.7,-48.4,-46.7,-48.4,-41.9,26.7,-19.4,-50.0,-58.1
4,-48.4,-14.3,-41.9,13.3,-35.5,-43.3,-61.3,-19.4,43.3,-12.9,-13.3,-51.6
5,-32.3,-39.3,-32.3,0.0,-3.2,-16.7,-48.4,-16.1,30.0,-12.9,-33.3,-61.3
6,-29.0,-21.4,-38.7,6.7,25.8,26.7,-12.9,9.7,20.0,-19.4,-43.3,-54.8
7,-41.9,-25.0,-35.5,33.3,16.1,-23.3,-38.7,-9.7,26.7,-38.7,-60.0,-74.2
8,-16.1,-21.4,-51.6,-6.7,9.7,-53.3,-41.9,-29.0,20.0,-45.2,-66.7,-87.1


### 2022

In [13]:
direction2022 = (
    100*res_smp_direction.loc[2022].reset_index().set_index("Ay").T
).style.background_gradient(
    cmap="RdYlGn",axis=None
).format("{:,.1f}").set_caption('<br>2022 yılı Sistem Yönü Ay Bazında Saatlik Ortalamalar<br>Enerji Fazlası: 100,<br> Dengede: 0<br> Enerji Açığı: -100')

direction2022

Ay,1,2,3,4,5,6,7,8,9,10,11,12
Saat,,,,,,,,,,,,
0,-48.4,-35.7,-80.6,-66.7,-29.0,-40.0,-61.3,-74.2,-56.7,-35.5,-46.7,-35.5
1,-67.7,-50.0,-93.5,-56.7,-3.2,-46.7,-61.3,-93.5,-66.7,-48.4,-40.0,-58.1
2,-64.5,-71.4,-74.2,-33.3,3.2,-53.3,-48.4,-87.1,-66.7,-38.7,-43.3,-48.4
3,-41.9,-67.9,-74.2,-53.3,9.7,-43.3,-48.4,-83.9,-56.7,-29.0,-46.7,-51.6
4,-32.3,-67.9,-90.3,-53.3,0.0,-33.3,-32.3,-80.6,-53.3,-25.8,-43.3,-54.8
5,-48.4,-82.1,-87.1,-33.3,-3.2,-26.7,-41.9,-80.6,-60.0,-38.7,-50.0,-45.2
6,-67.7,-82.1,-93.5,26.7,6.5,-6.7,-16.1,-67.7,-63.3,-41.9,-53.3,-51.6
7,-54.8,-78.6,-87.1,13.3,9.7,-36.7,-35.5,-54.8,-60.0,-61.3,-70.0,-67.7
8,-61.3,-78.6,-87.1,-40.0,-29.0,-30.0,-41.9,-67.7,-80.0,-54.8,-76.7,-87.1


### 2023

In [14]:
direction2023 = (
    100*res_smp_direction.loc[2023].reset_index().set_index("Ay").T
).style.background_gradient(
    cmap="RdYlGn",axis=None
).format("{:,.1f}").set_caption('<br>2023 yılı Sistem Yönü Ay Bazında Saatlik Ortalamalar<br>Enerji Fazlası: 100,<br> Dengede: 0<br> Enerji Açığı: -100')

direction2023

Ay,1,2,3,4,5,6,7,8,9,10,11,12
Saat,,,,,,,,,,,,
0,-48.4,-42.9,-74.2,-53.3,-67.7,-53.3,-67.7,-61.3,66.7,3.2,26.7,29.0
1,-80.6,-78.6,-87.1,-66.7,-54.8,-80.0,-74.2,-54.8,40.0,0.0,-26.7,16.1
2,-87.1,-64.3,-67.7,-40.0,-67.7,-66.7,-80.6,-25.8,26.7,0.0,-3.3,29.0
3,-67.7,-78.6,-74.2,-13.3,-51.6,-76.7,-74.2,-41.9,36.7,6.5,6.7,35.5
4,-61.3,-78.6,-58.1,-36.7,-48.4,-73.3,-67.7,-41.9,40.0,12.9,0.0,22.6
5,-67.7,-85.7,-67.7,-60.0,-48.4,-80.0,-80.6,-41.9,36.7,12.9,3.3,12.9
6,-58.1,-78.6,-67.7,-53.3,-35.5,-46.7,-74.2,-45.2,-10.0,-41.9,-16.7,0.0
7,-61.3,-82.1,-22.6,-20.0,-48.4,-53.3,-54.8,-48.4,33.3,-54.8,-63.3,-19.4
8,-80.6,-71.4,-61.3,-26.7,-67.7,-66.7,-80.6,-35.5,3.3,-22.6,-80.0,-45.2


### Export MCP

In [15]:
dfi.export(mcp2021, 'mcp_2021.png', dpi=400)
dfi.export(mcp2022, 'mcp_2022.png', dpi=400)
dfi.export(mcp2023, 'mcp_2023.png', dpi=400)

dfi.export(direction2021, 'direction_2021.png', dpi=400)
dfi.export(direction2022, 'direction_2022.png', dpi=400)
dfi.export(direction2023, 'direction_2023.png', dpi=400)
